## Part II: Machine Learning and Business Use Case

Now your manager has a basic understanding of why customers returned orders. Next, he wants you to use machine learning to predict which orders are most likely to be returned. In this part, you will generate several features based on our previous findings and your manager's requirements.

### Problem 4: Feature Engineering
#### Step 1: Create the dependent variable
- First of all, we need to generate a categorical variable which indicates whether an order has been returned or not.
- ***Hint:*** the returned orders’ IDs are contained in the dataset “returns”

In [72]:
import pandas as pd
orders = pd.read_csv('../data/Orders.csv')
returns = pd.read_csv('../data/Returns.csv')

In [73]:
orders_ret = orders.merge(right=returns, how='left', left_on="Order.ID", right_on="Order ID") # merge orders and returns dataframes
orders_ret['Returned_d'] = pd.get_dummies(orders_ret['Returned'])['Yes']  # new feature: Returned_d -> 1 = Yes 0 = No

#### Step 2:
- Your manager believes that **how long it took the order to ship** would affect whether the customer would return it or not. 
- He wants you to generate a feature which can measure how long it takes the company to process each order.
- ***Hint:*** Process.Time = Ship.Date - Order.Date

In [74]:
orders_ret['Ship.Date'] = pd.to_datetime(orders_ret['Ship.Date'])    # converting dates to datetime
orders_ret['Order.Date'] = pd.to_datetime(orders_ret['Order.Date'])  # converting dates to datetime
orders_ret['Process.Time'] = orders_ret['Ship.Date'] - orders_ret['Order.Date']   # new feature: Process.Time  = Ship.Date - Order.Date

#### Step 3:

- If a product has been returned before, it may be returned again. 
- Let us generate a feature indicates how many times the product has been returned before.
- If it never got returned, we just impute using 0.
- ***Hint:*** Group by different Product.ID


In [75]:
number_returns = pd.DataFrame((orders_ret[orders_ret['Returned_d']==1]).groupby('Product.ID').size()) # calculates how many time a Product.ID was returned
number_returns.reset_index(inplace=True)  # reset index -> the Product.ID is now a column
number_returns.columns = ['Product.ID', 'Returned.count'] # rename the column names
order_ret = orders_ret.merge(number_returns, how="left", on="Product.ID") # merge number of returns with order_ret dataframe
order_ret['Returned.count'].fillna(0, inplace=True) # replace NaN with zero

### Problem 5: Fitting Models

- You can use any binary classification method you have learned so far.
- Use 80/20 training and test splits to build your model. 
- Double check the column types before you fit the model.
- Only include useful features. i.e all the `ID`s should be excluded from your training set.
- Note that there are only less than 5% of the orders have been returned, so you should consider using the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-80/topics/createDataPartition) function from `caret` package and [StratifiedKfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn-model-selection-stratifiedkfold) from sklearn when running cross-validation.
- Do forget to `set.seed()` before the spilt to make your result reproducible.
- **Note:** We are not looking for the best tuned model in the lab so don't spend too much time on grid search. Focus on model evaluation and the business use case of each model.


In [129]:
pd.set_option('display.max_columns', 500)

In [132]:
# clean up
# orders_ret.drop[columns=[orders_ret.filter(regex='\S.ID$').columns]]

column_names = []
for i in orders_ret.filter(regex='\S.ID$'): # list of columns with ID in the name
    column_names.append(i)

for i in orders_ret.filter(regex='\S.Date$'): # list of columns with Date in the name
    column_names.append(i)

for i in orders_ret.filter(regex='\S.Name$'): # list of columns with Name in the name
    column_names.append(i)   

order_ret.drop(columns=column_names)

,Ship.Mode,Segment,Postal.Code,City,State,Country,Region_x,Market,Category,Sub.Category,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Returned,Region_y,Returned_d,Process.Time,Returned.count
0,First Class,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,Technology,Phones,$221.98,2,0.0,$62.15,40.770,High,NaN,NaN,0,2 days,0.0
1,Second Class,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,Furniture,Chairs,"$3,709.40",9,0.1,-$288.77,923.630,Critical,NaN,NaN,0,2 days,2.0
2,First Class,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,Technology,Phones,"$5,175.17",9,0.1,$919.97,915.490,Medium,NaN,NaN,0,1 days,2.0
3,First Class,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,Technology,Phones,"$2,892.51",5,0.1,-$96.54,910.160,Medium,NaN,NaN,0,2 days,0.0
4,Same Day,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,Technology,Copiers,"$2,832.96",8,0.0,$311.52,903.040,Critical,NaN,NaN,0,1 days,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,Same Day,Corporate,NaN,Kure,Hiroshima,Japan,Eastern Asia,Asia Pacific,Office Supplies,Fasteners,$65.10,5,0.0,$4.50,1.010,Medium,NaN,NaN,0,0 days,1.0
51286,Standard Class,Consumer,37421.0,Chattanooga,Tennessee,United States,Southern US,USCA,Furniture,Furnishings,$16.72,5,0.2,$3.34,1.930,High,NaN,NaN,0,4 days,0.0
51287,Second Class,Consumer,94109.0,San Francisco,California,United States,Western US,USCA,Office Supplies,Art,$8.56,2,0.0,$2.48,1.580,High,NaN,NaN,0,5 days,0.0
51288,Standard Class,Home Office,NaN,Valinhos,São Paulo,Brazil,South America,LATAM,Office Supplies,Binders,$13.44,2,0.0,$2.40,1.003,Medium,NaN,NaN,0,4 days,2.0


In [121]:
order_ret.columns

Index(['Row.ID', 'Order.ID', 'Order.Date', 'Ship.Date', 'Ship.Mode',
       'Customer.ID', 'Customer.Name', 'Segment', 'Postal.Code', 'City',
       'State', 'Country', 'Region_x', 'Market', 'Product.ID', 'Category',
       'Sub.Category', 'Product.Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping.Cost', 'Order.Priority', 'Returned', 'Order ID',
       'Region_y', 'Returned_d', 'Process.Time', 'Returned.count'],
      dtype='object')


### Problem 6: Evaluating Models
- What is the best metric to evaluate your model. Is accuracy good for this case?
- Now you have multiple models, which one would you pick? 
- Can you get any clue from the confusion matrix? What is the meaning of precision and recall in this case? Which one do you care the most? How will your model help the manager make decisions?
- **Note:** The last question is open-ended. Your answer could be completely different depending on your understanding of this business problem.

### Problem 7: Feature Engineering Revisit
- Is there anything wrong with the new feature we generated? How should we fix it?
- ***Hint***: For the real test set, we do not know it will get returned or not.